## 1️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Idempotent Google Drive mount
if not os.path.isdir('/content/drive/MyDrive'):
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted!")
else:
    print("✅ Google Drive already mounted.")

# Create output directory in Google Drive
GDRIVE_OUTPUT = '/content/drive/MyDrive/Udemy_Courses'
os.makedirs(GDRIVE_OUTPUT, exist_ok=True)

print(f"📁 Courses will be saved to: {GDRIVE_OUTPUT}")

## 2️⃣ Install System Dependencies

In [ ]:
%%bash

echo "📦 Installing system dependencies..."

# Update package list
apt-get update -qq

# Install ffmpeg
echo "Installing ffmpeg..."
apt-get install -y -qq ffmpeg > /dev/null 2>&1

# Install aria2
echo "Installing aria2..."
apt-get install -y -qq aria2 > /dev/null 2>&1

# Download and install shaka-packager
echo "Installing shaka-packager..."
wget -q https://github.com/shaka-project/shaka-packager/releases/download/v2.6.1/packager-linux-x64 -O /usr/local/bin/shaka-packager
chmod +x /usr/local/bin/shaka-packager

echo "✅ System dependencies installed!"

## 3️⃣ Clone Repository & Install Python Dependencies

In [ ]:
# Clone the repository (replace URL with yours)
import os

REPO_URL = "https://github.com/PerspicaciousGuy/Advance-Udemy-Downloader.git"
DEST_DIR = "/content/udemy-downloader"

if not os.path.exists(DEST_DIR):
    print("📥 Cloning repository...")
    os.system(f"git clone {REPO_URL} {DEST_DIR}")
else:
    print("✅ Repository already exists at", DEST_DIR)

# Change to project directory
os.chdir(DEST_DIR)
print(f"📁 Changed working directory to: {os.getcwd()}")

In [ ]:
# Install Python dependencies
print("📦 Installing Python packages...")
!pip install -q requests beautifulsoup4 m3u8 browser-cookie3 pysrt webvtt-py \
    coloredlogs python-dotenv pathvalidate tqdm lxml yt-dlp demoji bitstring protobuf

print("✅ Python dependencies installed!")

## 4️⃣ Upload Cookie and Key Files

**Important:** You need to prepare these files first:

### 📝 `cookie.txt`
1. Install [Cookies Editor](https://cookie-editor.com/) browser extension
2. Log in to Udemy in your browser
3. Open Cookies Editor and export in **Netscape** format
4. Save as `cookie.txt`

### 🔑 `keyfile.json`
1. Install [Widevine L3 Decrypter](https://addons.mozilla.org/en-US/firefox/addon/widevine-l3-decrypter/) on Firefox
2. Play any video from the course
3. Open the extension and click "Guess"
4. Copy the Key ID and Key
5. Create `keyfile.json` like this:
```json
{
  "KEY_ID_HERE": "KEY_HERE"
}
```

In [ ]:
from google.colab import files
import shutil, os

cookie_target = '/content/udemy-downloader/cookie.txt'
if os.path.exists(cookie_target):
    print(f"✅ cookie.txt already present at {cookie_target}. Skipping upload.")
else:
    print("📤 Upload your cookie.txt file:")
    uploaded = files.upload()
    for filename in uploaded.keys():
        if 'cookie' in filename.lower():
            shutil.move(filename, cookie_target)
            print(f"✅ Cookie file uploaded: {filename}")
            break

In [ ]:
from google.colab import files
import shutil, os

key_target = '/content/udemy-downloader/keyfile.json'
if os.path.exists(key_target):
    print(f"✅ keyfile.json already present at {key_target}. Skipping upload.")
else:
    print("📤 Upload your keyfile.json file:")
    uploaded = files.upload()
    for filename in uploaded.keys():
        if 'key' in filename.lower() or filename.endswith('.json'):
            shutil.move(filename, key_target)
            print(f"✅ Key file uploaded: {filename}")
            break

## 5️⃣ Upload Project Files (If Not Cloned from Git)

If you didn't clone from GitHub, upload the main project files here:

In [ ]:
import os

# Check if main.py exists
if not os.path.exists('/content/udemy-downloader/main.py'):
    print("⚠️ Project files not found!")
    print("📤 Please upload all project files (.py files):")
    
    uploaded = files.upload()
    
    # Move uploaded files
    for filename in uploaded.keys():
        shutil.move(filename, f'/content/udemy-downloader/{filename}')
        print(f"✅ Uploaded: {filename}")
else:
    print("✅ Project files found!")

## 6️⃣ Configure Download Settings

In [ ]:
# ========== CONFIGURATION ==========

# Course URL (REQUIRED)
COURSE_URL = "https://www.udemy.com/course/YOUR-COURSE-NAME/learn"  # ⬅️ CHANGE THIS

# Download Options
DOWNLOAD_CAPTIONS = True          # Download subtitles
DOWNLOAD_ASSETS = True            # Download supplementary files
DOWNLOAD_QUIZZES = True           # Download quizzes
SKIP_LECTURES = False             # Set to True to skip video downloads
SUBSCRIPTION_COURSE = False       # Set to True if this is a subscription course

# Video Quality (leave empty for best quality)
VIDEO_QUALITY = ""                # e.g., "720" or "1080" or "" for best

# Caption Language
CAPTION_LANGUAGE = "en"           # "en" or "all" for all languages

# Concurrent Downloads (1-30)
CONCURRENT_DOWNLOADS = 15

# Chapter Filter (optional)
CHAPTERS = ""                     # e.g., "1,3-5,7" or "" for all chapters

# Advanced Options
USE_H265 = False                  # Re-encode to H.265 (smaller files, slower)
KEEP_VTT = False                  # Keep .vtt caption files
SKIP_HLS = False                  # Skip HLS streams (faster)
CONTINUOUS_NUMBERING = True       # Use continuous lecture numbers

# Output Directory (Google Drive)
OUTPUT_DIR = GDRIVE_OUTPUT

print("✅ Configuration set!")
print(f"📌 Course URL: {COURSE_URL}")
print(f"📁 Output: {OUTPUT_DIR}")

## 7️⃣ Start Download 🚀

In [ ]:
import subprocess
import os

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Change to project directory
os.chdir('/content/udemy-downloader')

# Build command
cmd = [
    "python", "main.py",
    "-c", COURSE_URL,
    "-o", OUTPUT_DIR,
    "-cd", str(CONCURRENT_DOWNLOADS)
]

# Add optional arguments
if DOWNLOAD_CAPTIONS:
    cmd.append("--download-captions")
    cmd.extend(["-l", CAPTION_LANGUAGE])

if DOWNLOAD_ASSETS:
    cmd.append("--download-assets")

if DOWNLOAD_QUIZZES:
    cmd.append("--download-quizzes")

if SKIP_LECTURES:
    cmd.append("--skip-lectures")

if VIDEO_QUALITY:
    cmd.extend(["-q", str(VIDEO_QUALITY)])

if CHAPTERS:
    cmd.extend(["--chapter", CHAPTERS])

if USE_H265:
    cmd.append("--use-h265")

if KEEP_VTT:
    cmd.append("--keep-vtt")

if SKIP_HLS:
    cmd.append("--skip-hls")

if CONTINUOUS_NUMBERING:
    cmd.append("-n")

# Subscription-course toggle (set SUBSCRIPTION_COURSE = True in the config cell to enable)
subscription_flag = globals().get("SUBSCRIPTION_COURSE", False)
if subscription_flag:
    cmd.append("--subscription-course")

# Print command and confirmation
print("🚀 Starting download...")
print("Subscription course:", subscription_flag)
print("📝 Command:", ' '.join(cmd))
print("="*50)

# Execute
try:
    subprocess.run(cmd, check=True)
    print("="*50)
    print("✅ Download completed!")
    print(f"📁 Check your files at: {OUTPUT_DIR}")
except subprocess.CalledProcessError as e:
    print("="*50)
    print(f"❌ Error occurred: {e}")
    print("Check the error messages above for details.")


## 6.1️⃣ Inline Cookie & Keys (No Upload)

Use this section to paste your Netscape `cookie.txt` content and define your `keyfile.json` values directly in the notebook. If provided, the upload prompts will be skipped automatically.

In [ ]:
# ========== AUTH CONFIG (EDIT HERE) ==========

# Option A: Inline values (preferred for Colab users)
# Paste Netscape-format cookies between the triple quotes.
COOKIE_NETSCAPE = """
# Example format (do not include this comment in your real cookies):
# .udemy.com	TRUE	/	TRUE	2147483647	access_token	YOUR_ACCESS_TOKEN
# .udemy.com	TRUE	/	TRUE	2147483647	client_id	YOUR_CLIENT_ID
"""

# Define Widevine KID->KEY map here. KID and KEY must be hex strings.
KEY_JSON = {
    # "KID_HEX": "KEY_HEX",
    # Example:
    # "a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4": "0123456789abcdef0123456789abcdef",
}

# Option B: Read auth files from Google Drive
# If you prefer storing credentials in Drive, point AUTH_DRIVE_FOLDER to
# a folder inside your Drive that contains `cookie.txt` and `keyfile.json`.
# Set AUTH_FROM_DRIVE = True to enable automatic copy from Drive.
AUTH_FROM_DRIVE = True
AUTH_DRIVE_FOLDER = '/content/drive/MyDrive/Udemy_Auth'  # change if you use a different folder
AUTH_OVERWRITE = False  # if True, Drive files will overwrite inline values/files

import os, json, shutil
project_dir = '/content/udemy-downloader'
os.makedirs(project_dir, exist_ok=True)

cookie_path = os.path.join(project_dir, 'cookie.txt')
key_path = os.path.join(project_dir, 'keyfile.json')

# 1) If inline COOKIE_NETSCAPE provided, write it (highest priority)
if COOKIE_NETSCAPE.strip():
    with open(cookie_path, 'w', encoding='utf-8') as f:
        f.write(COOKIE_NETSCAPE.strip())
    print(f"✅ Wrote cookie.txt at {cookie_path} from inline text")
else:
    # 2) If enabled, try to copy from Drive auth folder
    if AUTH_FROM_DRIVE and os.path.isdir(AUTH_DRIVE_FOLDER):
        drive_cookie = os.path.join(AUTH_DRIVE_FOLDER, 'cookie.txt')
        if os.path.exists(drive_cookie):
            if not os.path.exists(cookie_path) or AUTH_OVERWRITE:
                shutil.copy(drive_cookie, cookie_path)
                print(f"✅ Copied cookie.txt from {drive_cookie} to {cookie_path}")
            else:
                print(f"ℹ️ cookie.txt already exists at {cookie_path}, set AUTH_OVERWRITE=True to replace")
        else:
            print(f"⚠️ No cookie.txt in Drive auth folder: {AUTH_DRIVE_FOLDER}")
    else:
        if not os.path.exists(cookie_path):
            print("⚠️ COOKIE_NETSCAPE is empty and cookie.txt not found. You can still upload later.")

# 3) KEY_JSON inline
if KEY_JSON:
    with open(key_path, 'w', encoding='utf-8') as f:
        json.dump(KEY_JSON, f)
    print(f"✅ Wrote keyfile.json at {key_path} from inline KEY_JSON")
else:
    # 4) Try Drive folder for keyfile
    if AUTH_FROM_DRIVE and os.path.isdir(AUTH_DRIVE_FOLDER):
        drive_key = os.path.join(AUTH_DRIVE_FOLDER, 'keyfile.json')
        if os.path.exists(drive_key):
            if not os.path.exists(key_path) or AUTH_OVERWRITE:
                shutil.copy(drive_key, key_path)
                print(f"✅ Copied keyfile.json from {drive_key} to {key_path}")
            else:
                print(f"ℹ️ keyfile.json already exists at {key_path}, set AUTH_OVERWRITE=True to replace")
        else:
            print(f"⚠️ No keyfile.json in Drive auth folder: {AUTH_DRIVE_FOLDER}")
    else:
        if not os.path.exists(key_path):
            print("⚠️ KEY_JSON is empty and keyfile.json not found. You can still upload later.")

# Show final status
print('\nFinal files in project:')
print(' -', 'cookie.txt ->', 'FOUND' if os.path.exists(cookie_path) else 'MISSING')
print(' -', 'keyfile.json ->', 'FOUND' if os.path.exists(key_path) else 'MISSING')


## 8️⃣ View Course Info Only (Optional)

Run this instead of the download cell if you just want to see course information:

In [ ]:
import subprocess
import os

os.chdir('/content/udemy-downloader')

# View course info without downloading
!python main.py -c "{COURSE_URL}" --info

## 9️⃣ Download Multiple Courses (Optional)

Want to download multiple courses? Add them to the list below:

In [ ]:
import subprocess
import os
import time

# List of course URLs
COURSES = [
    "https://www.udemy.com/course/course-1/learn",
    "https://www.udemy.com/course/course-2/learn",
    "https://www.udemy.com/course/course-3/learn",
]

os.chdir('/content/udemy-downloader')

for i, course_url in enumerate(COURSES, 1):
    print(f"\n{'='*60}")
    print(f"📚 Downloading course {i}/{len(COURSES)}")
    print(f"🔗 URL: {course_url}")
    print(f"{'='*60}\n")
    
    cmd = [
        "python", "main.py",
        "-c", course_url,
        "-o", OUTPUT_DIR,
        "--download-captions",
        "--download-assets",
        "-cd", "15"
    ]
    
    try:
        subprocess.run(cmd, check=True)
        print(f"\n✅ Course {i} completed!\n")
    except subprocess.CalledProcessError as e:
        print(f"\n❌ Course {i} failed: {e}\n")
        continue
    
    # Small delay between courses
    if i < len(COURSES):
        time.sleep(5)

print("\n" + "="*60)
print("🎉 All courses processed!")
print(f"📁 Check your files at: {OUTPUT_DIR}")
print("="*60)

## 🔟 Troubleshooting

### Common Issues:

**"Could not find course data"**
- Make sure `cookie.txt` is valid and uploaded correctly
- Try re-exporting cookies from your browser

**"Key not found"**
- You need to extract Widevine keys using the browser extension
- Make sure `keyfile.json` is formatted correctly

**"Command not found"**
- Re-run the system dependencies installation cell

**Session timeout**
- Google Colab sessions last ~12 hours
- For large courses, download in batches using chapter filter

---

## ⚠️ Important Notes:

1. **Legal**: This is for educational purposes only. Downloading courses may violate Udemy's TOS.
2. **Time Limits**: Colab sessions have time limits. Large courses may need multiple sessions.
3. **Storage**: Make sure you have enough Google Drive storage space.
4. **Speed**: Download speed depends on Colab's network, which can vary.
5. **Keys**: DRM keys expire, so extract them fresh for each course.

---

## 💡 Tips:

- Use `--chapter "1-5"` to download in batches
- Set `SKIP_HLS = True` for faster processing
- Lower `CONCURRENT_DOWNLOADS` if you get errors
- Keep the Colab tab open to prevent disconnection